<a href="https://colab.research.google.com/github/Catherine-Nguyen88/project_voting/blob/main/merging_and_wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script for merging and data cleaning

In [1]:
# clone from repo
! git clone https://github.com/Catherine-Nguyen88/project_voting

Cloning into 'project_voting'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 175 (delta 41), reused 45 (delta 18), pack-reused 90
Receiving objects: 100% (175/175), 65.44 MiB | 6.68 MiB/s, done.
Resolving deltas: 100% (77/77), done.
Updating files: 100% (64/64), done.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.linear_model import LinearRegression

## Merging for estimates

In [3]:
voting_original = pd.read_csv('./project_voting/data/voting_VA.csv')
voting_original.head(5)

# modify voting CSV
def match_county_name(county_name):
  if county_name.endswith('CITY'):
    return county_name.title()
  else:
    return county_name.title() + ' County'

voting_df = voting_original.copy()
voting_df['county_name'] = voting_df['county_name'].apply(lambda x: match_county_name(x))
voting_df.head()

# now, for each county, get candidatevotes/totalvotes
voting_df['fractionalvotes'] = voting_df['candidatevotes']/voting_df['totalvotes']

# split voting data into separate years for merging
voting_2000 = voting_df[voting_df['year']==2000]
print(f'Years for voting_2000 {voting_2000["year"].unique()}')
voting_2004 = voting_df[voting_df['year']==2004]
print(f'Years for voting_2004 {voting_2004["year"].unique()}')
voting_2008 = voting_df[voting_df['year']==2008]
print(f'Years for voting_2008 {voting_2008["year"].unique()}')
voting_2012 = voting_df[voting_df['year']==2012]
print(f'Years for voting_2012 {voting_2012["year"].unique()}')
voting_2016 = voting_df[voting_df['year']==2016]
print(f'Years for voting_2016 {voting_2016["year"].unique()}')
voting_2020 = voting_df[voting_df['year']==2020]
print(f'Years for voting_2020 {voting_2020["year"].unique()}')

# looks correct

Years for voting_2000 [2000]
Years for voting_2004 [2004]
Years for voting_2008 [2008]
Years for voting_2012 [2012]
Years for voting_2016 [2016]
Years for voting_2020 [2020]


#### 2000 election

In [4]:
# extract demographic data
fname = './project_voting/data/county_data/0002_ds176_20105_county_E.csv'
dem = pd.read_csv(fname)
# need to drop the 1st row
dem = dem.drop([0])
dem_VA = dem[dem['STATE'] == 'Virginia']
print(dem_VA.shape) # the shape is correct for VA
# rename for merging
dem_VA = dem_VA.rename(columns={'COUNTY':'county_name'})

# merge the datasets
merged_2000 = voting_2000.merge(dem_VA, on='county_name', how='left')
merged_2000.head(5)

(0, 276)


,Unnamed: 0_x,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,...,JSDE003,JSDE004,JSDE005,JSDE006,JSDE007,JSDE008,JSDE009,JSDE010,JS5E001,JTIE001
0,11161,2000,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11162,2000,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11163,2000,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,RALPH NADER,GREEN,220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11164,2000,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,OTHER,OTHER,261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11165,2000,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 2004 election

In [5]:
# extract demographic data
fname = './project_voting/data/county_data/0002_ds176_20105_county_E.csv'
dem = pd.read_csv(fname)
# need to drop the 1st row
dem = dem.drop([0])
dem_VA = dem[dem['STATE'] == 'Virginia']
print(dem_VA.shape) # the shape is correct for VA
# rename for merging
dem_VA = dem_VA.rename(columns={'COUNTY':'county_name'})

# merge the datasets
merged_2004 = voting_2004.merge(dem_VA, on='county_name', how='left')
merged_2004.head(5)

(0, 276)


,Unnamed: 0_x,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,...,JSDE003,JSDE004,JSDE005,JSDE006,JSDE007,JSDE008,JSDE009,JSDE010,JS5E001,JTIE001
0,20838,2004,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,JOHN KERRY,DEMOCRAT,5518,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20839,2004,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,7726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20840,2004,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,OTHER,OTHER,112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20841,2004,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,JOHN KERRY,DEMOCRAT,22088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20842,2004,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,21189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 2008 election

In [6]:
# extract demographic data
fname = './project_voting/data/county_data/0002_ds191_20125_county_E.csv'
dem = pd.read_csv(fname)
# need to drop the 1st row
dem = dem.drop([0])
dem_VA = dem[dem['STATE'] == 'Virginia']
print(dem_VA.shape) # the shape is correct for VA
# rename for merging
dem_VA = dem_VA.rename(columns={'COUNTY':'county_name'})

# merge the datasets
merged_2008 = voting_2008.merge(dem_VA, on='county_name', how='left')
merged_2008.head(5)

(134, 191)


,Unnamed: 0_x,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,...,QXSE007,QX6E001,QX7E001,QX7E002,QX7E003,QX8E001,QX8E002,QX8E003,QZTE001,QZ6E001
0,30189,2008,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,BARACK OBAMA,DEMOCRAT,7607,...,10973,21017,21017,14286,6731,14286,10070,4216,741,153800
1,30190,2008,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,JOHN MCCAIN,REPUBLICAN,7833,...,10973,21017,21017,14286,6731,14286,10070,4216,741,153800
2,30191,2008,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,OTHER,OTHER,183,...,10973,21017,21017,14286,6731,14286,10070,4216,741,153800
3,30192,2008,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,BARACK OBAMA,DEMOCRAT,29792,...,30576,42332,42332,37549,4783,37549,24648,12901,1080,332400
4,30193,2008,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,JOHN MCCAIN,REPUBLICAN,20576,...,30576,42332,42332,37549,4783,37549,24648,12901,1080,332400


#### 2012 election

In [7]:
# extract demographic data
fname = './project_voting/data/county_data/0002_ds206_20145_county_E.csv'
dem = pd.read_csv(fname)
# need to drop the 1st row
dem = dem.drop([0])
dem_VA = dem[dem['STATE'] == 'Virginia']
print(dem_VA.shape) # the shape is correct for VA
# rename for merging
dem_VA = dem_VA.rename(columns={'COUNTY':'county_name'})

# merge the datasets
merged_2012 = voting_2012.merge(dem_VA, on='county_name', how='left')
merged_2012.head(5)

(133, 192)


,Unnamed: 0_x,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,...,ABGVE001,ABGWE001,ABGWE002,ABGWE003,ABGXE001,ABGXE002,ABGXE003,ABIHE001,ABIOE001,ABITE001
0,39540,2012,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,BARACK OBAMA,DEMOCRAT,7655,...,21054,21054,14289,6765,14289,10053,4236,715,26.2,152500
1,39541,2012,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,MITT ROMNEY,REPUBLICAN,8213,...,21054,21054,14289,6765,14289,10053,4236,715,26.2,152500
2,39542,2012,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,OTHER,OTHER,183,...,21054,21054,14289,6765,14289,10053,4236,715,26.2,152500
3,39543,2012,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,BARACK OBAMA,DEMOCRAT,29757,...,43128,43128,38537,4591,38537,25135,13402,1115,30.0,317300
4,39544,2012,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,MITT ROMNEY,REPUBLICAN,23297,...,43128,43128,38537,4591,38537,25135,13402,1115,30.0,317300


#### 2016

In [8]:
# extract demographic data
fname = './project_voting/data/county_data/0002_ds225_20165_county_E.csv'
dem = pd.read_csv(fname)
# need to drop the 1st row
dem = dem.drop([0])
dem_VA = dem[dem['STATE'] == 'Virginia']
print(dem_VA.shape) # the shape is correct for VA
# rename for merging
dem_VA = dem_VA.rename(columns={'COUNTY':'county_name'})

# merge the datasets
merged_2016 = voting_2016.merge(dem_VA, on='county_name', how='left')
merged_2016.head(5)

(133, 187)


,Unnamed: 0_x,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,...,AF67E003,AF67E004,AF67E005,AF67E006,AF67E007,AF7PE001,AF7PE002,AF7PE003,AF89E001,AF9LE001
0,48891,2016,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,HILLARY CLINTON,DEMOCRAT,6740,...,15206,14137,1069,149,11575,13819,9605,4214,749,151900
1,48892,2016,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,DONALD TRUMP,REPUBLICAN,8583,...,15206,14137,1069,149,11575,13819,9605,4214,749,151900
2,48893,2016,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,OTHER,OTHER,495,...,15206,14137,1069,149,11575,13819,9605,4214,749,151900
3,48894,2016,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,HILLARY CLINTON,DEMOCRAT,33345,...,51563,49890,1673,354,33063,39431,25584,13847,1156,317300
4,48895,2016,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,DONALD TRUMP,REPUBLICAN,19259,...,51563,49890,1673,354,33063,39431,25584,13847,1156,317300


#### 2020 election

In [9]:
# extract demographic data
fname = './project_voting/data/county_data/0002_ds249_20205_county_E.csv'
dem = pd.read_csv(fname)
# need to drop the 1st row
dem = dem.drop([0])
dem_VA = dem[dem['STATE'] == 'Virginia']
print(dem_VA.shape) # the shape is correct for VA
# rename for merging
dem_VA = dem_VA.rename(columns={'COUNTY':'county_name'})

# merge the datasets
merged_2020 = voting_2020.merge(dem_VA, on='county_name', how='left')
merged_2020.head(5)

<ipython-input-9-49f0410600a6>:3: DtypeWarning: Columns (4,5,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267

(133, 993)


,Unnamed: 0_x,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,...,AMWSE004,AMWSE005,AMWSE006,AMWSE007,AMWSE008,AMWSE009,AMWSE010,AMWSE011,AMWSE012,AMWSE013
0,70379,2020,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,5495,...,9929,8310,2429,6248,1300,1444,102,10,414,3225
1,70380,2020,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,2072,...,9929,8310,2429,6248,1300,1444,102,10,414,3225
2,70381,2020,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,11,...,9929,8310,2429,6248,1300,1444,102,10,414,3225
3,70382,2020,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,JO JORGENSEN,LIBERTARIAN,66,...,9929,8310,2429,6248,1300,1444,102,10,414,3225
4,70383,2020,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,JO JORGENSEN,LIBERTARIAN,122,...,9929,8310,2429,6248,1300,1444,102,10,414,3225


In [10]:
#Combining all year voting data
dfs = [merged_2000, merged_2004, merged_2008, merged_2012, merged_2016, merged_2020]

#Finding common columns
com_cols = set.intersection(*[set(df.columns) for df in dfs])

#Same columns
common = [df[list(com_cols)] for df in dfs]

#Combining along row
VA_estimates0 = pd.concat(common, ignore_index=True)
print(VA_estimates0.shape)
VA_estimates0.head()

#Counting number of NA's
na_counts = VA_estimates0.isna().sum()
print(na_counts)

#Dropping columns where all values are NA's
VA_estimates = VA_estimates0.dropna(axis=1, how='all')
print(VA_estimates.shape)

VA_estimates.head()

(3736, 54)
STATE              1780
GISJOIN            1780
RES_ONLYA          3736
AIHHTLI            3736
ANRCA              3736
state                 0
METDIVA            3736
candidate             0
mode                  0
office                0
BLKGRPA            3736
COUNTYA            1780
YEAR               1780
PCI                3736
CNECTAA            3736
TRACTA             3736
Unnamed: 0_x          0
CBSAA              3736
SDSECA             3736
BTBGA              3736
NECTADIVA          3736
fractionalvotes       3
CDCURRA            3736
TRUSTA             3736
UAA                3736
totalvotes            0
party                 0
candidatevotes        0
state_po              0
Unnamed: 0_y       1780
SDUNIA             3736
PUMAA              3736
version               0
STATEA             1780
county_fips           0
AIANHHA            3736
CSAA               3736
CONCITA            3736
STUSAB             1780
SDELMA             3736
AITSCEA            3736
COUSU

,STATE,GISJOIN,state,candidate,mode,office,COUNTYA,YEAR,Unnamed: 0_x,fractionalvotes,...,state_po,Unnamed: 0_y,version,STATEA,county_fips,STUSAB,GEOID,NAME,year,county_name
0,NaN,NaN,VIRGINIA,AL GORE,TOTAL,US PRESIDENT,NaN,NaN,11161,0.427002,...,VA,NaN,20220315,NaN,51001,NaN,NaN,NaN,2000,Accomack County
1,NaN,NaN,VIRGINIA,GEORGE W. BUSH,TOTAL,US PRESIDENT,NaN,NaN,11162,0.532662,...,VA,NaN,20220315,NaN,51001,NaN,NaN,NaN,2000,Accomack County
2,NaN,NaN,VIRGINIA,RALPH NADER,TOTAL,US PRESIDENT,NaN,NaN,11163,0.018449,...,VA,NaN,20220315,NaN,51001,NaN,NaN,NaN,2000,Accomack County
3,NaN,NaN,VIRGINIA,OTHER,TOTAL,US PRESIDENT,NaN,NaN,11164,0.021887,...,VA,NaN,20220315,NaN,51001,NaN,NaN,NaN,2000,Accomack County
4,NaN,NaN,VIRGINIA,AL GORE,TOTAL,US PRESIDENT,NaN,NaN,11165,0.441161,...,VA,NaN,20220315,NaN,51003,NaN,NaN,NaN,2000,Albemarle County


In [11]:
print(VA_estimates.columns)

Index(['STATE', 'GISJOIN', 'state', 'candidate', 'mode', 'office', 'COUNTYA',
       'YEAR', 'Unnamed: 0_x', 'fractionalvotes', 'totalvotes', 'party',
       'candidatevotes', 'state_po', 'Unnamed: 0_y', 'version', 'STATEA',
       'county_fips', 'STUSAB', 'GEOID', 'NAME', 'year', 'county_name'],
      dtype='object')


## Fix table names for dfs

In [14]:
names2010 = pd.read_csv('./project_voting/ds17620105Ecodebook.csv')
names2010['NHGIS'] = names2010['NHGIS'].str.rstrip(':')
names2010['NHGIS'] = names2010['NHGIS'].str.lstrip()
names2010['Name'] = names2010['Name'].str.lstrip()
names2010_dict = names2010.set_index('NHGIS')['Name'].to_dict()

names2012 = pd.read_csv('./project_voting/ds19120125Ecodebook.csv')
names2012['NHGIS'] = names2012['NHGIS'].str.rstrip(':')
names2012['NHGIS'] = names2012['NHGIS'].str.lstrip()
names2012['Name'] = names2012['Name'].str.lstrip()
names2012_dict = names2012.set_index('NHGIS')['Name'].to_dict()

names2014 = pd.read_csv('./project_voting/ds20620145Ecodebook.csv')
names2014['NHGIS'] = names2014['NHGIS'].str.rstrip(':')
names2014['NHGIS'] = names2014['NHGIS'].str.lstrip()
names2014['Name'] = names2014['Name'].str.lstrip()
names2014_dict = names2014.set_index('NHGIS')['Name'].to_dict()

names2016 = pd.read_csv('./project_voting/ds22520165Ecodebook.csv')
names2016['NHGIS'] = names2016['NHGIS'].str.rstrip(':')
names2016['NHGIS'] = names2016['NHGIS'].str.lstrip()
names2016['Name'] = names2016['Name'].str.lstrip()
names2016_dict = names2016.set_index('NHGIS')['Name'].to_dict()

names2020 = pd.read_csv('./project_voting/ds24920205Ecodebook.csv')
names2020['NHGIS'] = names2020['NHGIS'].str.rstrip(':')
names2020['NHGIS'] = names2020['NHGIS'].str.lstrip()
names2020['Name'] = names2020['Name'].str.lstrip()
names2020_dict = names2020.set_index('NHGIS')['Name'].to_dict()

In [19]:
dfs2 = pd.concat(dfs, ignore_index=True)
dfs2.rename(columns=names2010_dict,inplace=True)
dfs2.rename(columns=names2012_dict,inplace=True)
dfs2.rename(columns=names2014_dict,inplace=True)
dfs2.rename(columns=names2016_dict,inplace=True)
dfs2.rename(columns=names2020_dict,inplace=True)

In [25]:
pd.set_option('display.max_columns', None)
dfs2

Unnamed: 0_x  year     state state_po       county_name  county_fips  \
0            11161  2000  VIRGINIA       VA   Accomack County        51001   
1            11162  2000  VIRGINIA       VA   Accomack County        51001   
2            11163  2000  VIRGINIA       VA   Accomack County        51001   
3            11164  2000  VIRGINIA       VA   Accomack County        51001   
4            11165  2000  VIRGINIA       VA  Albemarle County        51003   
...            ...   ...       ...      ...               ...          ...   
3731         71966  2020  VIRGINIA       VA   Winchester City        51840   
3732         71967  2020  VIRGINIA       VA   Winchester City        51840   
3733         71968  2020  VIRGINIA       VA   Winchester City        51840   
3734         71969  2020  VIRGINIA       VA   Winchester City        51840   
3735         71970  2020  VIRGINIA       VA   Winchester City        51840   

            office       candidate       party  candidatevotes  totalvotes  \
0     US PRESIDENT         AL GORE    DEMOCRAT            5092       11925   
1     US PRESIDENT  GEORGE W. BUSH  REPUBLICAN            6352       11925   
2     US PRESIDENT     RALPH NADER       GREEN             220       11925   
3     US PRESIDENT           OTHER       OTHER             261       11925   
4     US PRESIDENT         AL GORE    DEMOCRAT           16255       36846   
...            ...             ...         ...             ...         ...   
3731  US PRESIDENT           OTHER       OTHER              24       12113   
3732  US PRESIDENT           OTHER       OTHER               0       12113   
3733  US PRESIDENT  DONALD J TRUMP  REPUBLICAN            2825       12113   
3734  US PRESIDENT  DONALD J TRUMP  REPUBLICAN            2370       12113   
3735  US PRESIDENT  DONALD J TRUMP  REPUBLICAN              26       12113   

       version          mode  fractionalvotes  Unnamed: 0_y  X.2  X.1    X  \
0     20220315         TOTAL         0.427002           NaN  NaN  NaN  NaN   
1     20220315         TOTAL         0.532662           NaN  NaN  NaN  NaN   
2     20220315         TOTAL         0.018449           NaN  NaN  NaN  NaN   
3     20220315         TOTAL         0.021887           NaN  NaN  NaN  NaN   
4     20220315         TOTAL         0.441161           NaN  NaN  NaN  NaN   
...        ...           ...              ...           ...  ...  ...  ...   
3731  20220315  ELECTION DAY         0.001981           NaN  NaN  NaN  NaN   
3732  20220315   PROVISIONAL         0.000000           NaN  NaN  NaN  NaN   
3733  20220315      ABSENTEE         0.233221           NaN  NaN  NaN  NaN   
3734  20220315  ELECTION DAY         0.195658           NaN  NaN  NaN  NaN   
3735  20220315   PROVISIONAL         0.002146           NaN  NaN  NaN  NaN   

     GIS Join Match Code  NaN  NaN Region Code Division Code  NaN  NaN  \
0                    NaN  NaN  NaN         NaN           NaN  NaN  NaN   
1                    NaN  NaN  NaN         NaN           NaN  NaN  NaN   
2                    NaN  NaN  NaN         NaN           NaN  NaN  NaN   
3                    NaN  NaN  NaN         NaN           NaN  NaN  NaN   
4                    NaN  NaN  NaN         NaN           NaN  NaN  NaN   
...                  ...  ...  ...         ...           ...  ...  ...   
3731                 NaN  NaN  NaN         NaN           NaN  NaN  NaN   
3732                 NaN  NaN  NaN         NaN           NaN  NaN  NaN   
3733                 NaN  NaN  NaN         NaN           NaN  NaN  NaN   
3734                 NaN  NaN  NaN         NaN           NaN  NaN  NaN   
3735                 NaN  NaN  NaN         NaN           NaN  NaN  NaN   

     County Code County Subdivision Code  NaN  NaN Block Group Code  \
0            NaN                     NaN  NaN  NaN              NaN   
1            NaN                     NaN  NaN  NaN              NaN   
2            NaN                     NaN  NaN  NaN              NaN   
3            NaN      